In [1]:
import argparse
import io

from IPython.core.display import display, HTML
import IPython.display as ipd


from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

from pydub import AudioSegment

import os
from scipy.io import wavfile
from scipy.fftpack import fft
import librosa

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'HeedBook-59df779e8e5a.json'

In [3]:
client = speech.SpeechClient()

In [4]:
from pymongo import MongoClient
client = MongoClient()

In [5]:
def play_mp3(audio):
    audio.export('./export/current.wav', format='wav')
    audio, rt = librosa.load('./export/current.wav')
    display(ipd.Audio(audio, rate=rt))

In [6]:
def export_to_raw(audio):
    audio.export('./export/to_raw.wav', 'wav')
    os.system('sox -v 0.99 ./export/to_raw.wav -b 16 ./alchemist/input.raw channels 1 rate 16000')
    return './alchemist/input.raw'

In [7]:
import json

In [8]:
def transcribe_file(speech_file):
    
    out = []
    
    client = speech.SpeechClient()

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='ru-RU',
        enable_word_time_offsets=True)
    
    response = client.recognize(config, audio)
    transcript = ''
    for result in response.results:
        alternative = result.alternatives[0]
        transcript = alternative.transcript
        #print('Transcript: {}'.format(alternative.transcript))

        for word_info in alternative.words:
            curr_word = {}
            word = word_info.word
            curr_word['Word'] = word
            start_time = word_info.start_time
            end_time = word_info.end_time
            curr_word['Begin'] = start_time.seconds + start_time.nanos * 1e-9
            curr_word['End'] = end_time.seconds + end_time.nanos * 1e-9
            out.append(curr_word)
            answ = 'Word: {}, start_time: {}, end_time: {}'.format(
                word,
                start_time.seconds + start_time.nanos * 1e-9,
                end_time.seconds + end_time.nanos * 1e-9)
            
            #print(answ)
    return response, out, transcript

In [9]:
path = './alchemist/0001.mp3'

In [10]:
song = AudioSegment.from_mp3(path)

In [11]:
play_mp3(song[:20000])

In [12]:
x, curr_list, transcript = transcribe_file(export_to_raw(song[:20000]))

In [13]:
import alignment

In [14]:
transcript

'Пауло Коэльо Алхимик Однажды пречистая Дева с младенцем Христом на руках спустилась на землю и посетил некую монашеская Обитель исполненные гордости монахи выстроились в ряд'

In [15]:
curr_list

[{'Begin': 0.0, 'End': 0.8, 'Word': 'Пауло'},
 {'Begin': 0.8, 'End': 1.1, 'Word': 'Коэльо'},
 {'Begin': 1.1, 'End': 1.5, 'Word': 'Алхимик'},
 {'Begin': 1.5, 'End': 6.7, 'Word': 'Однажды'},
 {'Begin': 6.7, 'End': 7.3, 'Word': 'пречистая'},
 {'Begin': 7.3, 'End': 7.6, 'Word': 'Дева'},
 {'Begin': 7.6, 'End': 8.5, 'Word': 'с'},
 {'Begin': 8.5, 'End': 8.6, 'Word': 'младенцем'},
 {'Begin': 8.6, 'End': 9.1, 'Word': 'Христом'},
 {'Begin': 9.1, 'End': 9.5, 'Word': 'на'},
 {'Begin': 9.5, 'End': 9.6, 'Word': 'руках'},
 {'Begin': 9.6, 'End': 10.8, 'Word': 'спустилась'},
 {'Begin': 10.8, 'End': 10.9, 'Word': 'на'},
 {'Begin': 10.9, 'End': 11.0, 'Word': 'землю'},
 {'Begin': 11.0, 'End': 11.3, 'Word': 'и'},
 {'Begin': 11.3, 'End': 12.8, 'Word': 'посетил'},
 {'Begin': 12.8, 'End': 13.2, 'Word': 'некую'},
 {'Begin': 13.2, 'End': 13.8, 'Word': 'монашеская'},
 {'Begin': 13.8, 'End': 13.9, 'Word': 'Обитель'},
 {'Begin': 13.9, 'End': 16.1, 'Word': 'исполненные'},
 {'Begin': 16.1, 'End': 16.7, 'Word': 'горд

In [16]:
s = '0000fewfwe12000'

In [17]:
s.strip('0')

'fewfwe12'